In [1]:
from pytket.circuit import Circuit, CircBox, QControlBox, Unitary2qBox, DiagonalBox

from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus

from pytket.backends.backendresult import BackendResult
from pytket.extensions.qiskit import AerStateBackend
from pytket.passes import DecomposeBoxes

from jax import numpy as jnp
from jax import random, jit, Array
import phayes as bys
from typing import Tuple, Union, Callable
from datetime import datetime

import scipy as sp
import scipy.stats as sts
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
# Constructs unitary for an arbitrary size of qubits
def construct_unitary(n_qubits: int, theta) -> Circuit():
    ### Wait on Carson and Christine ###
    u_circ = Circuit(n_qubits)
    u_circ.U1(theta, 0)
    return u_circ

In [3]:
def create_diagonal_arr(n):
    N = int(math.pow(2, n))
    return [np.exp(2*math.pi*1j*theta/N) for theta in range(0, N)]
    
def build_u_matrix(beta: float, k: int):
    P = np.array([[1, 0], 
                  [0, np.exp(1.j * np.pi / 4)]])
    Rz = np.array([[np.exp(- 0.5j * beta), 0], 
                   [0, np.exp(0.5j * beta)]])
    U = np.tensordot(P, Rz, axes =0)
    U_k = np.linalg.matrix_power(U, k)
    return  U

def Rz(theta):
    return jnp.array([[np.exp(- 0.5j * theta), 0], [0, jnp.exp(0.5j * theta)]])

def build_phase_estimation_circuit(
    m: int, beta: float, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", 1)
    state_prep_register = qpe_circ.add_q_register("p", n_state_prep_qubits)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Add Hadamard gate to the measurement register
    qpe_circ.H(measurement_register[0])
    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)
    # Run the controlled unitary m times
    for _ in range(m):
        qpe_circ.add_qcontrolbox(
            controlled_u_gate, list(measurement_register) + list(state_prep_register)
        )

    qpe_circ.U1(beta/np.pi, measurement_register[0])
    qpe_circ.H(measurement_register[0])
    qpe_circ.measure_register(measurement_register, "c")
    return qpe_circ

In [4]:
def get_next_digit(m: int, beta: float, state_prep_circuit: Circuit, unitary_circuit: Circuit,
                   backend=AerStateBackend(), n_shots=100):
    qpe_circ = build_phase_estimation_circuit(m, beta, state_prep_circuit, unitary_circuit)
    backend_name = "bayes_qpe"
    state = 0
    p = 2
    configuration = QuantinuumConfig(device_name="H1-1E")

    DecomposeBoxes().apply(qpe_circ)


    compiled_circ = backend.get_compiled_circuit(qpe_circ)
    result = backend.run_circuit(compiled_circ, n_shots)
    sorted_shots = result.get_counts().most_common()
    print("Depth this run: ", qpe_circ.depth())
    print("Gates this run: ", qpe_circ.n_gates)
    return sorted_shots[0][0][0]

In [5]:
# Prepares circuit by notting all the qubits
def prepare_circ(circ: Circuit) -> Circuit():
    n_qubits = circ.n_qubits
    prepped_circ = Circuit(n_qubits)
    
    for i in range(n_qubits):
        prepped_circ.X(i)
    return prepped_circ

In [6]:
def bayes_iterator(prior_state: jnp.ndarray, n_shots: int, random_key: Array) -> jnp.ndarray:
    k, beta = jit(bys.get_k_and_beta)(prior_state)
    shots = gen_circs(k, beta, n_shots, random_key)
    posterior_state = jit(bys.update)(prior_state, shots, k, beta)
    return beta, k, posterior_state
    

In [7]:
def likelihood_func(datum, mu):
  likelihood_out = sts.norm.pdf(datum, mu) #Note that mu here is an array of values
  return likelihood_out/likelihood_out.sum()
    
def calc_beta(digits):
    beta = 0
    print(digits)
    mu = np.linspace(0, np.pi, num = len(digits))
    uniform_dist = sts.uniform.pdf(mu) + 1 #normalizing probabilities
    uniform_dist = uniform_dist/uniform_dist.sum()
    
    likelihood_out = likelihood_func(digits, mu)
    unnormalized_posterior = likelihood_out * uniform_dist
    for j in range(len(digits)):
            beta = beta - np.pi*digits[j]/2**(j+1)
    return beta

In [8]:
def iterated_qpe(state_prep_circuit: Circuit, unitary_circuit: Circuit, precision: int, backend):
    digits = []
    for i in range(precision-1,-1,-1):
        beta = calc_beta(digits)
        """
        for j in range(len(digits)):
            #beta = beta - np.pi*digits[j]/2**(j+1)
            #beta = beta - np.pi*digits[j]/2
        """
        x = get_next_digit(2**i, beta, state_prep_circuit, unitary_circuit, backend=backend)
        digits.insert(0,x)
        prop_correct = x / 100
        print("Recorded Correct: ", x)
    bitstring = "".join([str(bit) for bit in digits])
    print(bitstring)
    integer_j = int(bitstring, 2)

    # Calculate theta estimate
    return integer_j / (2 ** len(bitstring))

In [9]:
# MAIN 

test_qubits = 2
theta = np.pi / 4

fourier_coes = 50
states = [bys.init(fourier_coes)]
ks = []
betas = []


test_circ = Circuit(test_qubits)
prepped_circ = prepare_circ(test_circ)

test_u = construct_unitary(test_qubits, theta)
estimated_phase = iterated_qpe(prepped_circ, test_u, 5, backend)

print("Estimated Phase:", estimated_phase)
print("True Phase:", theta / 2)

error = round(abs(theta - (2 * estimated_phase)), 3)
print("Error:",error)

TypeError: iterated_qpe() missing 1 required positional argument: 'backend'

In [10]:
### Harmonic Oscillator Benchmarking
p = 2
precision = 6
bayesian_phase_est = Nexus().new_project(f"Bayesian Phase Estimation - Harmonic Oscillator - p: {p}, prec: {precision}, {datetime.now()}")

configuration = QuantinuumConfig(device_name="H1-1E")
quantinuum_backend = NexusBackend(
    backend_config= configuration, 
    project= bayesian_phase_est
)

def create_diagonal_arr(n):
    N = int(math.pow(2, n))
    return [np.exp(2*math.pi*1j*theta/N) for theta in range(0, N)]
prep_circuit = Circuit(p)

diagonal_box = DiagonalBox(create_diagonal_arr(p))
unitary_circuit = Circuit(p).add_diagonal_box(diagonal_box, range(p))

estimated_phase = iterated_qpe(prep_circuit, unitary_circuit, precision=precision,backend = quantinuum_backend)

print("Estimated Phase:", estimated_phase)
print("True Phase:", 0)


New project created: Bayesian Phase Estimation - Harmonic Oscillator - p: 2, prec: 6, 2024-02-04 13:24:32.865623

Started using project with name: Bayesian Phase Estimation - Harmonic Oscillator - p: 2, prec: 6, 2024-02-04 13:24:32.865623
[]
Depth this run:  356
Gates this run:  388
Recorded Correct:  1
[1]
Depth this run:  180
Gates this run:  196
Recorded Correct:  0
[0, 1]
Depth this run:  92
Gates this run:  100
Recorded Correct:  0
[0, 0, 1]
Depth this run:  48
Gates this run:  52
Recorded Correct:  0
[0, 0, 0, 1]
Depth this run:  26
Gates this run:  28
Recorded Correct:  0
[0, 0, 0, 0, 1]
Depth this run:  15
Gates this run:  16
Recorded Correct:  0
000001
Estimated Phase: 0.015625
True Phase: 0


In [ ]:
prop_correct = result.get_counts().most_common()[0][1] / 100